In [1]:
!pip install mediapipe opencv-python

                                              0.0/50.2 MB ? eta -:--:--
                                              0.1/50.2 MB 1.7 MB/s eta 0:00:29
                                              0.3/50.2 MB 3.8 MB/s eta 0:00:14
                                              1.0/50.2 MB 7.9 MB/s eta 0:00:07
     -                                        1.4/50.2 MB 8.9 MB/s eta 0:00:06
     -                                        2.2/50.2 MB 10.1 MB/s eta 0:00:05
     -                                        2.3/50.2 MB 8.7 MB/s eta 0:00:06
     --                                       3.4/50.2 MB 11.3 MB/s eta 0:00:05
     ---                                      4.1/50.2 MB 11.9 MB/s eta 0:00:04
     ----                                     5.3/50.2 MB 12.9 MB/s eta 0:00:04
     ----                                     5.8/50.2 MB 13.4 MB/s eta 0:00:04
     -----                                    6.6/50.2 MB 13.6 MB/s eta 0:00:04
     -----                                    7.3/50.

In [36]:
import cv2
import mediapipe as mp
import numpy as np
import threading
import time
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

[x: 0.5596819
y: 0.54296684
z: -1.3124115
visibility: 0.99155664
, x: 0.5946774
y: 0.3763707
z: -1.3463299
visibility: 0.97941417
, x: 0.621213
y: 0.36750615
z: -1.3457597
visibility: 0.9816569
, x: 0.6469582
y: 0.35906097
z: -1.3465412
visibility: 0.9809193
, x: 0.51730096
y: 0.37590444
z: -1.2836179
visibility: 0.9832359
, x: 0.49278075
y: 0.36656427
z: -1.2820061
visibility: 0.982853
, x: 0.4692719
y: 0.35769498
z: -1.2823868
visibility: 0.9811383
, x: 0.6854381
y: 0.33119345
z: -0.9943243
visibility: 0.98132575
, x: 0.44124517
y: 0.33359194
z: -0.66964
visibility: 0.9849997
, x: 0.6099224
y: 0.6200608
z: -1.1450082
visibility: 0.99423593
, x: 0.5165661
y: 0.6322231
z: -1.0487821
visibility: 0.9938759
, x: 0.92759407
y: 0.6827831
z: -0.6529486
visibility: 0.99644464
, x: 0.2395302
y: 0.80525935
z: -0.29640946
visibility: 0.998713
, x: 1.0479792
y: 1.0086933
z: -0.435915
visibility: 0.2102941
, x: 0.12960792
y: 1.4299823
z: -0.23206572
visibility: 0.12751251
, x: 1.155107
y: 1.812014

33

In [37]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [30]:
calculate_angle(shoulder, elbow, wrist)

165.86477582286022

([0.8569191694259644, 0.7440389394760132],
 [0.9884451627731323, 1.06656014919281],
 [1.0912821292877197, 1.7936152219772339])

In [77]:
cap = cv2.VideoCapture(0)

counter = 0 
stage = "down"


with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get left coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate left angle
            left_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            
            # Get right coordinates
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate right angle
            right_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            
          
            # Bench counter logic
            if left_angle > 160 and right_angle >160:
                stage = "up"
            if (left_angle < 60 and stage =='up') and right_angle<60:
                stage="down"
                counter +=1
                print(counter)
                
        except:
            pass
        
        # Render bench counter
        # Setup status box
        cv2.rectangle(image, (0,0), (1000,73), (0, 128, 0), -1)
        
        # Count Reps
        cv2.putText(image, 'REPS', (15,12),
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_COMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Up or Down
        cv2.putText(image, 'STAGE', (95,12), 
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (90,60), 
                    cv2.FONT_HERSHEY_COMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        #Calorie data
        cv2.putText(image, 'CALORIES BURNED', (285, 12),
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter*0.24), 
                    (280,60), 
                    cv2.FONT_HERSHEY_COMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(200,231,20), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(22,43,232), thickness=2, circle_radius=2) 
                                 )                       
        
        cv2.imshow('Based Stupid Gang', image)
        
        key = cv2.waitKey(10)
        if key & 0xFF == ord('r'):
            counter=0
            stage = None

        if key & 0xFF == ord('x'):
            break


    cap.release()
    cv2.destroyAllWindows()